In [5]:
import os 
import sys
import pandas as pd 
from sklearn.model_selection import train_test_split
from uvars import givemesomecredit_dir,tML_dir


In [6]:
sys.path.append(tML_dir)
from automl import agluon_pipeline

In [7]:
presents = ['medium_quality', 'good_quality','high_quality','best_quality']
problem_type = "binary" # ‘binary’, ‘multiclass’, ‘regression’, ‘quantile  
eval_metric = "roc_auc"
time_limit = 60 * 2#10 # 10 minutes 
verbosity = 1 
sample_weight = 'auto_weight'
expname = 'ag_def' # ag_tun
agluon_dir = os.path.join('submit', 'agluon',expname)
os.makedirs(agluon_dir, exist_ok=True)

presents_tun = ['high_quality','best_quality']
presents_def = ['medium_quality', 'good_quality','high_quality','best_quality'] 

exp_names = ['ag_def', 'ag_tun']
exp_presents = [presents_def,presents_tun]
assert len(exp_names) == len(exp_presents), 'list of iterators do not match'


datapth = os.path.join(givemesomecredit_dir)
train = pd.read_csv(os.path.join(datapth, 'cstraining.csv'))
test = pd.read_csv(os.path.join(datapth, 'cstest.csv'))
ss = pd.read_csv(os.path.join(datapth, 'cssampleentry.csv'))
data = [train, test]
for df in data:
    df.rename(columns={'Unnamed: 0':'Id'}, inplace=True)

del df 
tcol = 'SeriousDlqin2yrs'
fcols = train.drop(['Id',tcol], axis=1).columns.tolist()
X_train, X_test, y_train, y_test = train_test_split(train[fcols], train[tcol], test_size=0.1, stratify=train[tcol])
train_data = pd.concat([X_train, y_train], axis=1)
valid_data = pd.concat([X_test, y_test],axis=1)
train_data.shape, valid_data.shape

print(f"Unique classes: {list(train_data[tcol].unique())}")


Unique classes: [0, 1]


In [ ]:
for i in range(len(exp_names)) :
    expname = exp_names[i]
    presents = exp_presents[i]
    print(f'{expname} @{presents}')
    agluon_dir = os.path.join(datapth,'submit', 'agluon',expname)
    os.makedirs(agluon_dir, exist_ok=True)
    agluon_pipeline(
        expname=expname,
        agluon_dir=agluon_dir,
        presents=presents,
        problem_type=problem_type,
        eval_metric=eval_metric,
        verbosity=verbosity,
        sample_weight=sample_weight,
        train_data=train_data,
        valid_data=valid_data,
        test=test,
        fcols=fcols,
        tcol=tcol,
        time_limit=time_limit,
        calibrate_decision_threshold=True
    )